### Лематизация текстов (первые 4000 слов из каждого документа)

In [4]:
!pip install pymorphy2
!pip install nltk
!pip install gensim
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [5]:
import numpy as np
import pandas as pd
from tqdm import notebook
import re
from time import sleep
import os
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from tqdm import tqdm
import pickle
from scipy.sparse import csr_matrix
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool, Lock, Value
import functools
import operator
import requests
import pymorphy2
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
morph = pymorphy2.MorphAnalyzer()
from bs4.element import Comment
import os.path
from os import path

In [6]:
nltk.download("stopwords")
stop_words = list(set(stopwords.words('russian')))

In [7]:
morph = pymorphy2.MorphAnalyzer()
PYMORPHY_CACHE = {}
def lemmatizer(words):
    global PYMORPHY_CACHE
    for word in words:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [8]:
from tqdm import notebook
chunksize = 1
docs_latest_path = './docs_latest.tsv'
test_chunk = 0
if not os.path.exists('./docs'):
    os.makedirs('./docs')

for i, chunk in enumerate(notebook.tqdm(pd.read_csv(docs_latest_path, chunksize=1, header=None, sep='\t'))):
    test_chunk = chunk
    docwords = str(test_chunk[2].values[0]).lower().split()[:4000]
    docwords = [word for word in docwords if word not in stop_words]
    doc_lemmatized = ' '.join(lemmatizer(docwords))

    save_path = './docs/' + str(i)
    with open(save_path, 'w+') as f:
        f.write(doc_lemmatized)

### Лематизация заголовков

In [9]:
titles_path = './docs_latest_titles.tsv'
titles_df = pd.read_csv(titles_path, sep='\t', header=None)
titles_df.head()

In [10]:
titles_raw = titles_df.values[:, 1]
titles = []
for title in notebook.tqdm(titles_raw):
    words = str(title).lower().split()
    words = [word for word in words if word not in stop_words]
    titles.append(' '.join(lemmatizer(words)))

### Исправление запросов

In [11]:
import pandas as pd
queries_path = './ranking-long-tail-queries-fall-2020/queries.tsv'
bad_queries = pd.read_csv(queries_path, sep='\t', header=None)
bad_queries = bad_queries.values[:, 1]

In [12]:
import requests
queries = []
for query in notebook.tqdm(bad_queries):
    reply = requests.get('https://speller.yandex.net/services/spellservice.json/checkText?text=' + query)
    assert(reply.status_code == 200)
    suggest = {change['word']: change['s'][0] for change in reply.json() if len(change['s']) > 0}
    for word, correction in suggest.items():
        query = query.replace(word, correction)
    queries.append(query)

In [13]:
bad_queries[1] = queries
queries = bad_queries

### Bm25 по заголовкам и текстам

In [14]:
# coding: UTF-8
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency
#source: https://github.com/arosh/BM25Transformer
class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b : float, optional (default=0.75)
    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        """
        if not sp.issparse(X):
            X = sp.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf =   np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.spdiags(idf, diags=0, m=n_features, n=n_features)
        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.csr_matrix(X, dtype=np.float64, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        dl = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]
        # In each row, repeat `dl` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(dl), sz)
        # Average document length
        # Scalar value
        avgdl = np.average(dl)
        # Compute BM25 score only for non-zero elements
        data = X.data * (self.k1 + 1) / (X.data + self.k1 * (1 - self.b + self.b * rep / avgdl))
        data[data < 0] = 0
        X = sp.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            # *= doesn't work
            X = X * self._idf_diag

        return X

In [15]:
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer

class gen_title():
    def __iter__(self):
        for title in tqdm(titles):
            words = []
            for w in str(title).split():
                words.append(w.rstrip('\n'))
            yield words

vectorizer_h = CountVectorizer(tokenizer=lambda d: d, lowercase=False)

# sp_h ~= sparse header
sp_h = vectorizer_h.fit_transform(gen_title())

trans = BM25Transformer()
# sp_h_t ~= sparse header transformed
sp_h_t = trans.fit_transform(sp_h)

In [16]:
##texts
from tqdm import notebook

file_prefix = './docs/'
lemmatized_docs_filenames = sorted(os.listdir(file_prefix))
class gen_text():
    def __iter__(self):
        for docname in notebook.tqdm(lemmatized_docs_filenames):
            words = []
            with open(file_prefix + docname) as f:
                for w in str(f.read()).split():
                    words.append(w.rstrip('\n'))
                yield words

In [17]:
##texts
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_b = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

# sp_b ~= sparse body
sp_b = vectorizer_b.fit_transform(gen_text())

trans = BM25Transformer()
# sp_b_t ~= sparse body transformed
sp_b_t = trans.fit_transform(vectorizer_b)

In [18]:
sp_h_t, sp_b_t

In [19]:
term_to_idx_h = vectorizer_h.vocabulary_

In [20]:
marks_path = './ranking-long-tail-queries-fall-2020/train.marks.tsv/train.marks.tsv'
marks_df = pd.read_csv(marks_path, '\t', header=None)

tr_gr = dict()
for q_id, doc_id, _ in marks_df.values:
    if q_id not in tr_gr:
        tr_gr[q_id] = [doc_id]
    else:
        tr_gr[q_id].append(doc_id)
tr_q = queries[np.array(list(tr_gr.keys())), :]

In [21]:
sample_path = './ranking-long-tail-queries-fall-2020/sample.csv/sample.csv'
sample_subm = pd.read_csv(sample_path)

tt_gr = dict()
for q_id, doc_id in sample_subm.values:
    if q_id not in tt_gr:
        tt_gr[q_id] = [doc_id]
    else:
        tt_gr[q_id].append(doc_id)
tt_q = queries[np.array(list(tt_gr.keys())), :]

In [22]:
DocsId = []
QueryId = []

bm25title_scores = []
bm25text_scores = []

for q_id, query in notebook.tqdm(tr_q):
    q_split = str(query).split()

    idx = []
    for q_tok in q_split:
        if q_tok in term_to_idx_h:
            idx.append(term_to_idx_h[q_tok])
    idx = sorted(idx)
    scores_h = sp_h_t[:, idx].toarray().sum(axis=1)
    
    idx = []
    for q_tok in q_split:
        if q_tok in term_to_idx_b:
            idx.append(term_to_idx_b[q_tok])
    idx = sorted(idx)
    scores_b = sp_b_t[:, idx].toarray().sum(axis=1)
    
    scores_b = scores_h
    for doc_id, title_score, text_score in zip(tr_gr[q_id],
                                               scores_h,
                                               scores_b):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25title_scores.append(title_score)
        bm25text_scores.append(text_score)

In [23]:
train_df = pd.DataFrame({
    'QueryId' : QueryId,
    'DocumentId': DocsId,
    'bm25titles': bm25title_scores,
    'bm25first4000words': bm25text_scores
})

In [24]:
DocsId = []
QueryId = []

bm25title_scores = []
bm25text_scores = []

for q_id, query in notebook.tqdm(tt_q):
    q_split = str(query).split()
    
    idx = []
    for q_tok in q_split:
        if q_tok in term_to_idx_h:
            idx.append(term_to_idx_h[q_tok])
    idx = sorted(idx)
    scores_h = sp_h_t[:, idx].toarray().sum(axis=1)
    
    idx = []
    for q_tok in q_split:
        if q_tok in term_to_idx_b:
            idx.append(term_to_idx_b[q_tok])
    idx = sorted(idx)
    scores_b = sp_b_t[:, idx].toarray().sum(axis=1)
    
    scores_b = scores_h
    for doc_id, title_score, text_score in zip(tt_gr[q_id],
                                               scores_h,
                                               scores_b):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25title_scores.append(title_score)
        bm25text_scores.append(text_score)

In [25]:
test_df = pd.DataFrame({
    'QueryId' : QueryId,
    'DocumentId' : DocsId,
    'bm25titles': bm25title_scores,
    'bm25first4000words': bm25text_scores
})

### universal sentence encoder large по заголовкам

In [26]:
embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [27]:
titles_embds = []
for i in notebook.tqdm(range(len(titles))):
    current_emb = embed(str(titles[i]))
    titles_embds.append(current_emb)

In [28]:
query_embds = []
for i in notebook.tqdm(range(len(queries))):
    current_emb = embed(str(queries[i]))
    query_embds.append(current_emb)

In [29]:
tr_use_large = []
for q_id, query in notebook.tqdm(tr_q):
    for i in tr_gr[q_id]:
        current = cosine(query_embds[q_id].reshape(1, 1024), 
                         titles_embds[i].reshape(1, 1024))
        tr_use_large.append(current[0][0])

tt_use_large = []
for q_id, query in notebook.tqdm(tt_q):
    for i in tt_gr[q_id]:
        current = cosine(query_embds[q_id].reshape(1, 1024),
                         titles_embds[i].reshape(1, 1024))
        tt_use_large.append(current[0][0])

In [30]:
train_df['use_large'] = tr_use_large
test_df['use_large'] = tt_use_large
train_df.to_csv('tr_python_features', index=None)
test_df.to_csv('tt_python_features', index=None)